# Datas e horários

Ser capaz de lidar com datas e horas, especialmente para aplicações de séries temporais, é uma parte crucial da limpeza de dados. É fácil deixar que bugs sutis se instalem devido à análise incorreta de datas e horas ou à não consideração dos fusos horários. Em raras ocasiões, você pode encontrar conjuntos de dados provenientes de um sistema que corajosamente usa tipos de hora personalizados, como relógios de 27 horas (sim, isso aconteceu comigo!). A questão é que trabalhar com datas e horas pode ser confuso, então aprenderemos algumas estratégias práticas aqui.

In [ ]:
import pandas as pd 
import numpy as np 

Vamos carregar um dataframe do Github que contém 5 colunas de datas e horas, todas representando o mesmo valor, mas em 5 formatos diferentes.

In [ ]:
url = 'https://raw.githubusercontent.com/thomasnield/machine-learning-demo-data/master/timeseries/datetime_formatting.csv'

df = pd.read_csv(url)
df

Vamos extrair uma das colunas e analisar seus tipos de dados. Observe que se trata de um `dtype` de `object`, não de `datetime64` como gostaríamos.

In [ ]:
df['ORDER_DATE_TM1']

## Conversão implícita de data e hora

Se você quiser aplicar lógica útil de calendário ou baseada em tempo a esses valores, precisará convertê-los para um tipo de dado diferente. Uma maneira de fazer isso é chamar a função `to_datetime()` do Pandas nessa coluna. O Pandas fará o possível para analisar a data dessa série.

In [ ]:
parsed_col = pd.to_datetime(df['ORDER_DATE_TM1'])
parsed_col

Podemos usar lógica baseada em calendário para extrair propriedades, como o dia da semana.

In [ ]:
parsed_col.dt.dayofweek

Se você já sabe quais colunas deseja formatar como datas/horas, pode passar o parâmetro `parse_dates` para a função `read_csv()` com uma lista de nomes de colunas para as quais se espera o formato de datas/horas. Vamos analisar todas as datas usando essa abordagem e o resultado. Para resumir, vamos analisar apenas os três primeiros resultados.

In [ ]:
df_parsed = pd.read_csv(url, 
            parse_dates=['ORDER_DATE1','ORDER_DATE2','ORDER_DATE_TM1','ORDER_DATE_TM2','ORDER_DATE_TM3'])

df_parsed.head(3)

Há algum erro? Aliás, o `ORDER_DATE_TM2` foi analisado quase completamente errado! Por exemplo, o segundo registro realmente tem valores representando uma data e hora de `2022-09-25 20:16:00`, mas o `ORDER_DATE_TM2` foi analisado incorretamente como `2025-09-22 20:16:00`! O que aconteceu?

Bem, vamos analisar o valor original. Aliás, vamos amostrar os três primeiros registros e analisar o que está acontecendo.

In [ ]:
df.head(3)

O que parece estar acontecendo com `ORDER_DATE_TM2` é que ele está confundindo o dia e o mês. Convencionalmente, espera-se que um formato como `22-Set-25 8:16 PM` tenha o dia `22` no início, e foi isso que o Pandas presumiu. No entanto, um desenvolvedor sádico decidiu arbitrar sua própria convenção e registrar o ano nesse local, colocando o dia `25` após o mês.

Isso explica por que o primeiro registro `22-Jan-22 4:08PM` foi analisado corretamente, já que o ano e o dia do mês eram exatamente os mesmos.

Para lidar com isso, precisaremos fazer uma conversão explícita.

## Conversão explícita de data e hora

Estude as convenções de formatação de data e hora para Python aqui.

https://docs.python.org/3/library/datetime.html#strftime-and-strptime-behavior

`strftime()` e `strptime()` são usados para escrever uma data e hora em uma string de formatação e analisar uma data e hora a partir de uma string formatada, respectivamente. Os códigos de formatação vêm das convenções padrão de C. Aqui estão algumas comuns, muitas das quais usaremos neste notebook. Consulte o link acima para ver todos os códigos de formatação.

| Símbolo | Descrição | Exemplo de Análise: 2021-01-13 2:34PM |
|--------|-------------------------|----------------------------------|
| `%y` | Ano com 2 dígitos | 21 |
| `%m` | Mês com 2 dígitos | 01 |
| `%d` | Dia do mês com 2 dígitos | 13 |
| `%b` | Mês com 3 letras | Jan |
| `%I` | Hora para relógio de 12 horas | 2 |
| `%H` | Hora para relógio de 24 horas | 14 |
| `%M` | Minuto com 2 dígitos | 34 |
| `%p` | AM/PM para relógio de 12 horas | PM |
| `%S` | Segundos com 2 dígitos | 00 |
| `%f` | Microssegundos | 000000 |
| `%a` | Dia da semana com 3 letras | Qua |
| `%A` | Nome completo do dia da semana | Quarta-feira |

Para nossa coluna problemática `ORDER_DATE_TM2`, precisamos de `%y` para obter um ano com dois dígitos, `%b` para o nome do mês com três letras e `%d` para o dia do mês. Para o horário, usamos `%I` para a hora do relógio de 12 horas, `%M` para o minuto e `%p` para `AM/PM`. Corrigiremos a análise de `ORDER_DATE_TM2` passando-a para a função `to_datetime()` juntamente com o parâmetro `format` e, em seguida, atribuiremos isso de volta ao dataframe analisado.

In [ ]:
df_parsed['ORDER_DATE_TM2'] = pd.to_datetime(df['ORDER_DATE_TM2'], format='%y-%b-%d %I:%M %p')
df_parsed

Muito melhor! Agora todas as datas estão funcionando. Podemos verificar isso contando o número de linhas onde há apenas um valor único entre `ORDER_DATE_TM1`, `ORDER_DATE_TM2` e `ORDER_DATE_TM3`. De fato, todas as 1190 linhas têm exatamente um valor de data e hora único em cada linha.

In [ ]:
np.sum(df_parsed.loc[:,['ORDER_DATE_TM1','ORDER_DATE_TM2','ORDER_DATE_TM3']].nunique(axis=1) == 1)

Podemos fazer a mesma verificação para as duas colunas de datas simples para garantir que elas também foram analisadas corretamente.

In [ ]:
np.sum(df_parsed.loc[:,['ORDER_DATE1','ORDER_DATE2']].nunique(axis=1) == 1)

## Filtragem por data e hora

Vamos consolidar nosso dataframe e renomear as colunas apenas para `ORDER_DATE` e `ORDER_DATE_TM`

In [ ]:
df = df_parsed[['RECORD_ID','ORDER_DATE1','ORDER_DATE_TM1']] \
    .rename(columns={"ORDER_DATE1": "ORDER_DATE", "ORDER_DATE_TM1": "ORDER_DATE_TM"})

df

Digamos que queremos obter todos os registros cujo dia da semana seja terça-feira. Na propriedade `dt.dayofweek`, `0` será segunda-feira e `7` será domingo. Numericamente, isso fará com que terça-feira seja `1`. Podemos especificar isso como uma condição e retornar todos os registros que caem na terça-feira.

In [ ]:
condition = df['ORDER_DATE'].dt.dayofweek == 1
df[condition]

Você também pode filtrar rapidamente datas e horas usando um formato de string.

In [ ]:
df[df['ORDER_DATE'] >= '2022-06-01']

In [ ]:
df[('2022-06-01 12:35PM' <= df['ORDER_DATE_TM']) & (df['ORDER_DATE_TM'] <= '2022-06-05 8:05PM')]

Você também pode fazer conversões de data e hora mais explícitas para os limites inicial e final.

In [ ]:
start = pd.to_datetime('2022-06-01 12:35PM', format='%Y-%m-%d %I:%M%p')
end = pd.to_datetime('2022-06-05 8:05PM', format='%Y-%m-%d %I:%M%p')

df[(start <= df['ORDER_DATE_TM']) & (df['ORDER_DATE_TM'] <= end)]

## Lidando com fusos horários

Provavelmente, uma das maiores dores de cabeça que você pode encontrar na limpeza de dados quando se trata de datas e horas são as conversões de fuso horário. O ideal é que as datas e horas sejam armazenadas em **Tempo Universal Coordenado** ou UTC**, que é um padrão internacionalmente aceito para armazenamento de datas e horas. Se os horários precisarem ser convertidos localmente, **deslocamentos** são usados para expressar esse horário local com base no horário UTC. Isso parece mais fácil do que realmente é, porque as leis regionais ao redor do mundo evoluem e alteram os deslocamentos, especialmente devido ao horário de verão.

Felizmente, existe uma biblioteca conveniente chamada `pytz`, da qual o Pandas já depende. Ela cuidará dos deslocamentos de fuso horário, bem como do horário de verão, capturando até mesmo as leis de horário de verão alteradas no passado! Vamos importá-la e analisar os fusos horários comuns.

In [ ]:
import pytz

pytz.common_timezones

Uau! É muita coisa. Digamos que queremos localizar o horário do Arizona, nos Estados Unidos. O Arizona é meio especial porque não reconhece o horário de verão como o resto dos Estados Unidos. Recebe bastante luz solar o ano todo!

Vamos procurar pelo nome e salvá-lo em uma variável.

In [ ]:
tz = pytz.timezone('US/Arizona')
tz

Agora, vamos analisar a coluna `ORDER_DATE_TM` no nosso dataframe. Observe que não há informações sobre o fuso horário, o que o torna **ingênuo em relação ao fuso horário**.

In [ ]:
if df['ORDER_DATE_TM'].dt.tz is None:
    print("TZ NAIVE")
else:
    df['ORDER_DATE_TM'].dt.tz

Digamos que essas datas foram registradas no horário local do Arizona. Podemos tornar `ORDER_DATE_TM` sensível ao fuso horário chamando a função `tz_localize()` e especificando que elas foram registradas no horário `US/Arizona`.

In [ ]:
df['ORDER_DATE_TM'] = df['ORDER_DATE_TM'].dt.tz_localize('US/Arizona')
df['ORDER_DATE_TM']

Observe como o tipo de dado de `ORDER_DATE_TM` muda para `datetime64[ns, US/Arizona]`, deixando de ser um fuso horário nativo e passando a ser associado ao horário do Arizona. A parte `-07:00` do valor indica o deslocamento em relação ao UTC.

Agora podemos convertê-lo para diferentes fusos horários. Digamos que queremos adicionar uma coluna adicional `ORDER_DATE_TM_CST` mostrando a data e hora no horário dos EUA/Central. Podemos usar `tz_convert` para realizar essa conversão. Observe abaixo como o horário central às vezes está 2 horas à frente do Arizona, mas às vezes também está 1 hora à frente. Isso ocorre porque o Arizona não adota o horário de verão, mas o horário central sim.

In [ ]:
df['ORDER_DATE_TM_CST'] = df['ORDER_DATE_TM'].dt.tz_convert('US/Central')
df

Por fim, podemos, é claro, converter a data para `UTC`, que salvaremos na coluna `ORDER_DATE_TM_UTC`. Observe o deslocamento `+00:00`, pois UTC é a linha de base, sem deslocamento.

In [ ]:
df['ORDER_DATE_TM_UTC'] = df['ORDER_DATE_TM'].dt.tz_convert('UTC')
df

Não é ótimo o trabalho que a biblioteca `tz` fez por você? Recomendamos usá-la, pois ela manterá para você aquele banco de dados complexo de fusos horários, compensações, horário de verão e alterações históricas nas leis de horário de verão.

## EXERCÍCIO

Um dataframe de duas colunas e dois registros é mostrado abaixo. Complete o código abaixo substituindo os pontos de interrogação `?` para que `ORDER_DATE_TM` seja localizado para `US/Pacific`. Em seguida, adicione uma nova coluna `ORDER_DATE_TM_PARIS` que mostre o horário equivalente em `Europe/Paris`.

In [ ]:
import pandas as pd

df = pd.DataFrame({
    "ORDER_ID": [1, 2], 
    "ORDER_DATE_TM": [pd.to_datetime('2023-01-05 7:05 PM'), pd.to_datetime('2023-01-06 8:15 AM')]
})

# localize US/Pacific
df["ORDER_DATE_TM"] = df["ORDER_DATE_TM"].?

# converta para Europe/Paris 
df["ORDER_DATE_TM_PARIS"] = ?

df

### RESPOSTA A BAIXO

|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
v 

In [ ]:
import pandas as pd

df = pd.DataFrame({
    "ORDER_ID": [1, 2], 
    "ORDER_DATE_TM": [pd.to_datetime('2023-01-05 7:05 PM'), pd.to_datetime('2023-01-06 8:15 AM')]
})

# localize US/Pacific
df["ORDER_DATE_TM"] = df["ORDER_DATE_TM"].dt.tz_localize('US/Pacific')

# converta para Europe/Paris 
df["ORDER_DATE_TM_PARIS"] = df["ORDER_DATE_TM"].dt.tz_convert('Europe/Paris')

df